In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from keras.models import load_model
from numpy import array
import warnings
warnings.filterwarnings('ignore')
from pandas._libs import index
import plotly.express as px

In [2]:
#loading  data 
data= pd.read_csv("C:/LCF Official Website/core/ml_notebooks/combine.csv", parse_dates=['Date'], index_col='Date')

model1  = load_model('C:/LCF Official Website/core/ml_notebooks/model2.h5')

# **Currently available Data**

In [3]:
data
fig = px.line(data , y='waterLevel',   width=1500, height=1000, title='Groundwater level of Kuchlak from 2010 to 2020' , labels=dict(index="Date"))

fig.update_xaxes(
      rangeslider_visible=True)

fig.show()

In [4]:


def forecast(model, batch_size, data):
  test_predictions = []
  X = data.tail(12)
  Y = X.to_numpy()
  Y = Y.reshape(1, len(X), 1)

  for i in range(batch_size):
    
    # get the prediction value for the first batch
    current_pred = model1.predict(Y)[0]
    
    # append the prediction into the array
    test_predictions.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    Y = np.append(Y[:,1:,:],[[current_pred]],axis=1)
    

    def inner_func(X):  
        new = X
        new= pd.Index(pd.date_range(start = X.index[-1], periods= batch_size, freq = "M")).strftime('%Y-%m')
        new.index=  pd.to_datetime(new)
        return new
    

  
  df1 = pd.DataFrame(test_predictions,
                          index= inner_func(X),
                          columns=['waterlevel'])
  return df1




def plot(data):
  fig = px.line(df , y='waterlevel',   width=1500, height=1000, title=' Predicted Groundwater level of Kuchlak ' , labels=dict(index="Date"))

  fig.update_xaxes(
      rangeslider_visible=True,
      
      )

  return fig.show()



def func(data, siteID):
    well = data[siteID]
    #well= well[:102]
    well = well.tail(12)
    well = well.to_frame()
   
    return well
  

In [5]:
#batch size is actually the number of months you want to predict further

batch_size= int(input("Enter batch size   : "))
df= forecast(model1, batch_size, data)
plot(df)

1/1 [==============================] - 0s 18ms/step


# **Predictions by Site ID**

In [6]:
data2 = pd.read_csv("/content/drive/MyDrive/data/all_wells20.csv", parse_dates=['Date'] , index_col='Date')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/data/all_wells20.csv'

In [ ]:
site= input("Enter SITE ID  : ")
df1 = func(data2, site)
batch_size= int(input("Enter batch size   : "))
df= forecast(model1, batch_size, df1)
#plot(df)
df

Enter SITE ID  : K1
Enter batch size   : 12


,waterlevel
2020-12,1768.663818
2021-01,1768.013550
2021-02,1767.441650
2021-03,1766.637329
2021-04,1765.668335
2021-05,1764.719116
2021-06,1764.142822
2021-07,1763.551514
2021-08,1763.101318
2021-09,1762.492310
